# Export data to spreadsheet

Data in neo4j is exported to a spreadsheet for further analysis.

In neo4j `Respondent` has relationship `CHOSE_ALT` to `QuestionAlternative` which has relationship `CONSISTS_OF` from `Question`.

The `Respondent` has a property `id` which is used to identify the respondent in the spreadsheet.

The `QuestionAlternative` has a property `id` which is used to identify the question alternative in the spreadsheet.

The `QuestionAlternative` has a property `name` which is the text for the alternative.

The `QuestionAlternative` has a property `position` which is the position of the alternative in the question.

The `Question` has a property `id` which is used to identify the question in the spreadsheet.

The `Question` has a property `name` which is the text for the question.

The `Question` has a property `questionCategory` which is the category of the question.

## Connect to database

Should return version number if successful.


In [3]:
from jupyter_notebook.neo4j_setup.databaseconnection import DatabaseConnection

gds = DatabaseConnection().get_database_connection()
gds.version()

'2.3.2'

## Query data from database

In [10]:
# In neo4j `Respondent` has relationship `CHOSE_ALT` to `QuestionAlternative` which has relationship `CONSISTS_OF` from `Question`.

# The `Respondent` has a property `id` which is used to identify the respondent in the spreadsheet.

# The `QuestionAlternative` has a property `id` which is used to identify the question alternative in the spreadsheet.

# The `QuestionAlternative` has a property `name` which is the text for the alternative.

# The `QuestionAlternative` has a property `position` which is the position of the alternative in the question.

# The `Question` has a property `id` which is used to identify the question in the spreadsheet.

# The `Question` has a property `name` which is the text for the question.

# The `Question` has a property `questionType` which is the category of the question.

# The spreadsheet has the following sheets with the follwoing columns:
# Respondent: id, ...(one column for each Question, where the value is the pos for questionAlternative chosen)
# QuestionAlternative: id, name, position, questionId
# Question: id, name, questionCategory

result = gds.run_cypher("""
                MATCH (respondent:Respondent)-[:CHOSE_ALT]->(questionAlternative:QuestionAlternative)<-[:CONSISTS_OF]-(question:Question)
                RETURN 
                    respondent.id, respondent.schoolClimateNormalityIndex, respondent.victimizationNormalityIndex, respondent.louvainId_pos_prop, respondent.louvainId_similarity_pos_prop, respondent.lpaId_pos_prop, respondent.lpaId_similarity_pos_prop,
                    questionAlternative.id, questionAlternative.name, questionAlternative.position, 
                    question.id, question.name, question.questionCategories
                ORDER BY respondent.id, questionAlternative.id, questionAlternative.position
               """)

print(result.head())


                          respondent.id  \
0  00043e30-e387-44de-869b-ad46868697ff   
1  00043e30-e387-44de-869b-ad46868697ff   
2  00043e30-e387-44de-869b-ad46868697ff   
3  00043e30-e387-44de-869b-ad46868697ff   
4  00043e30-e387-44de-869b-ad46868697ff   

   respondent.schoolClimateNormalityIndex  \
0                                0.303766   
1                                0.303766   
2                                0.303766   
3                                0.303766   
4                                0.303766   

   respondent.victimizationNormalityIndex  respondent.louvainId_pos_prop  \
0                                0.522271                         1513.0   
1                                0.522271                         1513.0   
2                                0.522271                         1513.0   
3                                0.522271                         1513.0   
4                                0.522271                         1513.0   

   respondent

In [11]:
# Replace the `question.questionCategories` with the first item in the list of categories.
result['question.questionCategories'] = result['question.questionCategories'].apply(lambda x: x[0] if x else None)
print(result.head())

                          respondent.id  \
0  00043e30-e387-44de-869b-ad46868697ff   
1  00043e30-e387-44de-869b-ad46868697ff   
2  00043e30-e387-44de-869b-ad46868697ff   
3  00043e30-e387-44de-869b-ad46868697ff   
4  00043e30-e387-44de-869b-ad46868697ff   

   respondent.schoolClimateNormalityIndex  \
0                                0.303766   
1                                0.303766   
2                                0.303766   
3                                0.303766   
4                                0.303766   

   respondent.victimizationNormalityIndex  respondent.louvainId_pos_prop  \
0                                0.522271                         1513.0   
1                                0.522271                         1513.0   
2                                0.522271                         1513.0   
3                                0.522271                         1513.0   
4                                0.522271                         1513.0   

   respondent

## Generate spreadsheet from dataframe 

In [15]:
# In neo4j `Respondent` has relationship `CHOSE_ALT` to `QuestionAlternative` which has relationship `CONSISTS_OF` from `Question`.

# The `Respondent` has a property `id` which is used to identify the respondent in the spreadsheet.

# The `QuestionAlternative` has a property `id` which is used to identify the question alternative in the spreadsheet.

# The `QuestionAlternative` has a property `name` which is the text for the alternative.

# The `QuestionAlternative` has a property `position` which is the position of the alternative in the question.

# The `Question` has a property `id` which is used to identify the question in the spreadsheet.

# The `Question` has a property `name` which is the text for the question.

# The `Question` has a property `questionType` which is the category of the question.

# The spreadsheet has the following sheets with the follwoing columns:
# Respondent: id, ...(one column for each Question, where the value is the pos for questionAlternative chosen) 
# QuestionAlternative: id, name, position, questionId 
# Question: id, name, questionCategory

import shutil
import time
import pandas as pd

def create_new_community(data, community_column, min_community_size):
    # Get the number of students in each community
    community_sizes = data[community_column].value_counts()

    # Get the communities with less than 2 students
    small_communities = community_sizes[community_sizes < min_community_size]

    # Create a new community for all students in the small communities
    data.loc[data[community_column].isin(
        small_communities.index), community_column] = 'no community' if min_community_size == 2 else 'small community'

    return data


# Create a community for all students in communities with less than 2 students. Move all students in these communities to the new community. Do this for each of the three community detection algorithms.
# This is done to make the plot more readable.
result = create_new_community(result, 'respondent.louvainId_pos_prop', 2)
result = create_new_community(result, 'respondent.lpaId_pos_prop', 2)
result = create_new_community(result, 'respondent.louvainId_similarity_pos_prop', 2)
result = create_new_community(result, 'respondent.lpaId_similarity_pos_prop', 2)

# Create Respondent dataframe
respondent_df = result.pivot_table(index=['respondent.id', 'respondent.schoolClimateNormalityIndex', 'respondent.victimizationNormalityIndex', 'respondent.louvainId_pos_prop', 'respondent.louvainId_similarity_pos_prop', 'respondent.lpaId_pos_prop', 'respondent.lpaId_similarity_pos_prop'], columns='question.name', values='questionAlternative.name', aggfunc='first').reset_index()

# Create Question dataframe
question_df = result[['question.id', 'question.name', 'question.questionCategories', 'questionAlternative.id', 'questionAlternative.name', 'questionAlternative.position']].drop_duplicates()
# question_df = result.pivot_table(index=['question.id', 'question.name', 'question.questionCategories'], columns='questionAlternative.id', values='questionAlternative.name', aggfunc='first').reset_index()

# Create a copy of the Excel document
file_destination = f'results/spreadsheet_bjorn_{int(time.time())}.xlsx'
shutil.copyfile('results/spreadsheet_bjorn_1683728108.xlsx', file_destination)
print(f'New file created: {file_destination}')

# Export to Excel
with pd.ExcelWriter(file_destination, mode='a', if_sheet_exists='replace') as writer:
    respondent_df.to_excel(writer, index=False, sheet_name='Respondent')
    question_df.to_excel(writer, index=False, sheet_name='Question')

New file created: results/spreadsheet_bjorn_1683728378.xlsx
